# Machine Learning Engineer Nanodegree
## Capstone Project - Supervised Learning
## Project: Predict *Housing Price* of Russia under different macro environment

## Dataset Introduction 

Quote from https://www.kaggle.com/c/sberbank-russian-housing-market

Housing costs demand a significant investment from both consumers and developers. And when it comes to planning a budget—whether personal or corporate—the last thing anyone needs is uncertainty about one of their biggets expenses. Sberbank, Russia’s oldest and largest bank, helps their customers by making predictions about realty prices so renters, developers, and lenders are more confident when they sign a lease or purchase a building.

Although the housing market is relatively stable in Russia, the country’s volatile economy makes forecasting prices as a function of apartment characteristics a unique challenge. Complex interactions between housing features such as number of bedrooms and location are enough to make pricing predictions complicated. Adding an unstable economy to the mix means Sberbank and their customers need more than simple regression models in their arsenal.

In this competition, Sberbank is challenging Kagglers to develop algorithms which use a broad spectrum of features to predict realty prices. Competitors will rely on a rich dataset that includes housing data and macroeconomic patterns. An accurate forecasting model will allow Sberbank to provide more certainty to their customers in an uncertain economy.

In [1]:
#Basic Packages for Data Manipulation 
import numpy as np
import pandas as pd

#ML Models that will be used used
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.ensemble import GradientBoostingRegressor as gbr
from sklearn.ensemble import ExtraTreesRegressor as etr
from sklearn.svm import SVR as svr
from sklearn.linear_model import LinearRegression as lr
from xgboost import XGBRegressor as xgb
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Dropout

#Missing Value fill-in package
from sklearn.preprocessing import Imputer
from imputer import Imputer as KnnIm

#StandardScaler for Feature Scaling
from sklearn.preprocessing import StandardScaler

#Feature Reduction with PCA package
from sklearn.decomposition import PCA

# Cross-validation with GridSearch
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.model_selection import cross_val_score

# Time for record speed
import time

Using TensorFlow backend.
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/legacy/interfaces.py:89: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  wrapper._legacy_support_signature = inspect.getargspec(func)
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/legacy/interfaces.py:89: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  wrapper._legacy_support_signature = inspect.getargspec(func)
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/legacy/interfaces.py:89: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  wrapper._legacy_support_signature = inspect.getargspec(func)
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:212: DeprecationWarning: inspect.getargspec() is deprecated, u

/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/legacy/interfaces.py:89: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  wrapper._legacy_support_signature = inspect.getargspec(func)
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/legacy/interfaces.py:89: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  wrapper._legacy_support_signature = inspect.getargspec(func)
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/legacy/interfaces.py:89: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  wrapper._legacy_support_signature = inspect.getargspec(func)
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/legacy/interfaces.py:89: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getf

/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


----
## PART 1: Exploring the Data

First we are going to get the data from the dataset and pick some samples for a quick sense of how it look likes.

In [2]:
# Import supplementary visualization code visuals.py
import visuals as vs

# Pretty display for notebooks
%matplotlib inline

#Read dataset with pandas
train_data = pd.read_csv("train.csv", quoting = 2)
test_data = pd.read_csv("test.csv", quoting = 2)
macro_data = pd.read_csv("macro.csv", quoting = 2)

### Implementation: Data Exploration

Shape of the Dataset

In [3]:
print(train_data.shape)
print(test_data.shape)
print(macro_data.shape)

(30471, 292)
(7662, 291)
(2484, 100)


**Check out first few row of train / test / macro data for getting sense**

In [4]:
train_data.head(3).transpose()

,0,1,2
id,1,2,3
timestamp,2011-08-20,2011-08-23,2011-08-27
full_sq,43,34,43
life_sq,27,19,29
floor,4,3,2
max_floor,NaN,NaN,NaN
material,NaN,NaN,NaN
build_year,NaN,NaN,NaN
num_room,NaN,NaN,NaN
kitch_sq,NaN,NaN,NaN


In [5]:
test_data.head(3).transpose()

,0,1,2
id,30474,30475,30476
timestamp,2015-07-01,2015-07-01,2015-07-01
full_sq,39,79.2,40.5
life_sq,20.7,NaN,25.1
floor,2,8,3
max_floor,9,17,5
material,1,1,2
build_year,1998,0,1960
num_room,1,3,2
kitch_sq,8.9,1,4.8


In [6]:
macro_data.head(3).transpose()

,0,1,2
timestamp,2010-01-01,2010-01-02,2010-01-03
oil_urals,76.1,76.1,76.1
gdp_quart,NaN,NaN,NaN
gdp_quart_growth,NaN,NaN,NaN
cpi,NaN,NaN,NaN
ppi,NaN,NaN,NaN
gdp_deflator,NaN,NaN,NaN
balance_trade,NaN,NaN,NaN
balance_trade_growth,NaN,NaN,NaN
usdrub,NaN,NaN,NaN


**Observation:**
1. Data are separated in two files => Need to merge the two dataset (individual train data & macro environment data
2. Features with Categorial Values => Need to encode them into dummy variables for machine learning
3. Features with Missing Value => We need to fill-up the missing part given ANN doesn't support missing value

### Implementation: Data Preprocessing

Join different dataset into single X & remove useless info
1. Separate X, y from train data (price_doc is y while rest of features are X)
2. Given micro environment with unique id "timestamp", so we will join the macro data with train data by using "timestamp" 
3. Remove non-related data such as id & timestamp in column 0

In [7]:
train_data_y = train_data.iloc[:,-1]
train_data_X = train_data.iloc[:,1:-1]
test_data_X = test_data.iloc[:,1:]
train_data_X = pd.merge(train_data_X, macro_data, on='timestamp');
train_data_X = train_data_X.iloc[:,1:]
test_data_X = pd.merge(test_data_X, macro_data, on='timestamp');
test_data_X = test_data_X.iloc[:,1:]

Encode categorial features into dummy variables , to avoid new categorial values happening in testset not being captured, we will merge the dataset before converting

In [8]:
mixed_data_X = train_data_X.append(test_data_X, ignore_index=True)
length_train = len(train_data_X)
mixed_data_X = pd.get_dummies(mixed_data_X)

**Check out the data of mixed_data_X**

Shape of the mixed dataset

In [9]:
print(mixed_data_X.shape)

(38133, 557)


**Check out first few row of mixed data for getting sense**

In [10]:
mixed_data_X.head(3).transpose()

,0,1,2
full_sq,4.300000e+01,3.400000e+01,4.300000e+01
life_sq,2.700000e+01,1.900000e+01,2.900000e+01
floor,4.000000e+00,3.000000e+00,2.000000e+00
max_floor,NaN,NaN,NaN
material,NaN,NaN,NaN
build_year,NaN,NaN,NaN
num_room,NaN,NaN,NaN
kitch_sq,NaN,NaN,NaN
state,NaN,NaN,NaN
area_m,6.407578e+06,9.589337e+06,4.808270e+06


**Fill up missing value**

Check out which column has missing value and mark it in array

In [11]:
def findMissingValue(X):
    #Check out Empty Data
    EmptyDataList = []
    for i in range(0, len(X.iloc[0])):
        if(np.any(np.isnan(X.iloc[:,i]))):
            EmptyDataList.append(i)

    return EmptyDataList

Fill in Missing Value with different tactics (e.g. random forest = rfr, mean = take the mean of the column, knn = K nearest neighbour)

In [12]:
def fillMissingValue(df, fy, WaytoFillNan):
    train_data_temp = df[df.iloc[:,fy].notnull()]  
    test_data_temp = df[df.iloc[:,fy].isnull()]  
    train_y=train_data_temp.iloc[:,fy]
    train_X=train_data_temp.copy()
    train_X = train_X.drop(train_X.columns[fy], axis = 1)
    test_X = test_data_temp.copy()
    test_X = test_X.drop(test_X.columns[fy], axis =1)
    mixed_X = Imputer().fit_transform(train_X.append(test_X, ignore_index=True))
    length_train = len(train_X)
    train_X = mixed_X[:length_train,:]
    test_X = mixed_X[length_train:,:]
    
    if (WaytoFillNan == 'rfr'):
        print ("Try to fill-up value with rfr")
        rfr_regressor=rfr(n_estimators=100, verbose = 5, n_jobs = -1)
        rfr_regressor.fit(train_X,train_y)
        y_pred = rfr_regressor.predict(test_X)
        print (y_pred)
        df.iloc[:,fy] = df.iloc[:,fy].fillna(value = pd.Series(data = y_pred))

    elif (WaytoFillNan == 'mean'):
        print ("Try to fill-up value with mean")
        df.iloc[:,fy] = df.iloc[:,fy].fillna(value = np.mean(df.iloc[:,fy]))
        
    elif (WaytoFillNan == 'ffill'):
        df.iloc[:,fy] = df.iloc[:,fy].fillna(method = 'ffill')

    elif (WaytoFillNan == 'bfill'):
        df.iloc[:,fy] = df.iloc[:,fy].fillna(method = 'bfill')

    elif (WaytoFillNan == 'knn'):
        impute = KnnIm()
        df.iloc[:,fy] = (impute.knn(X=df, column=fy, k=10))[:,fy]
    
    return df

Fill up column by column and save to hdd per column to avoid running all over again if we stop in the middle

In [13]:
emptyList_mix = findMissingValue(mixed_data_X)

for i in emptyList_mix:
    print ("Filling-up column : " + str(i))
    mixed_data_X = fillMissingValue(mixed_data_X, i, 'knn')
    if(np.isnan(np.min(mixed_data_X.iloc[:,i])) == False):
        train_data_X = mixed_data_X.iloc[:length_train,:]
        test_data_X = mixed_data_X.iloc[length_train:,:]
        pd.DataFrame(train_data_X).to_csv("./training/train_data_X_for_capstone.csv", index=False)
        pd.DataFrame(test_data_X).to_csv("./training/test_data_X_for_capstone.csv", index=False)
        print ("Wrote to hdd til column : " + str(i))

Filling-up column : 1
Wrote to hdd til column : 1
Filling-up column : 2
Wrote to hdd til column : 2
Filling-up column : 3
Wrote to hdd til column : 3
Filling-up column : 4
Wrote to hdd til column : 4
Filling-up column : 5
Wrote to hdd til column : 5
Filling-up column : 6
Wrote to hdd til column : 6
Filling-up column : 7
Wrote to hdd til column : 7
Filling-up column : 8
Wrote to hdd til column : 8
Filling-up column : 14
Wrote to hdd til column : 14
Filling-up column : 17
Wrote to hdd til column : 17
Filling-up column : 20
Wrote to hdd til column : 20
Filling-up column : 55
Wrote to hdd til column : 55
Filling-up column : 56
Wrote to hdd til column : 56
Filling-up column : 57
Wrote to hdd til column : 57
Filling-up column : 58
Wrote to hdd til column : 58
Filling-up column : 59
Wrote to hdd til column : 59
Filling-up column : 60
Wrote to hdd til column : 60
Filling-up column : 61
Wrote to hdd til column : 61
Filling-up column : 62
Wrote to hdd til column : 62
Filling-up column : 63
Wrote

Safe check on whether all the columns have been filled up or not

In [17]:
print (np.any(np.isnan(mixed_data_X)))
print (np.any(np.isnan(train_data_X)))
print (np.any(np.isnan(test_data_X)))
print (len(train_data_X))
print (len(test_data_X))

False
False
False
30471
7662


**Feature Scaling**
Improve the speed of ANN with standard scaling

In [18]:
sc_X = StandardScaler()
train_data_X = sc_X.fit_transform(train_data_X)
test_data_X = sc_X.transform(test_data_X)

----
## PART 2: Measurement of Model

### Metrics and the Naive Predictor
*CharityML*, equipped with their research, knows individuals that make more than \$50,000 are most likely to donate to their charity. Because of this, *CharityML* is particularly interested in predicting who makes more than \$50,000 accurately. It would seem that using **accuracy** as a metric for evaluating a particular model's performace would be appropriate. Additionally, identifying someone that *does not* make more than \$50,000 as someone who does would be detrimental to *CharityML*, since they are looking to find individuals willing to donate. Therefore, a model's ability to precisely predict those that make more than \$50,000 is *more important* than the model's ability to **recall** those individuals. We can use **F-beta score** as a metric that considers both precision and recall:

$$ F_{\beta} = (1 + \beta^2) \cdot \frac{precision \cdot recall}{\left( \beta^2 \cdot precision \right) + recall} $$

In particular, when $\beta = 0.5$, more emphasis is placed on precision. This is called the **F$_{0.5}$ score** (or F-score for simplicity).

Looking at the distribution of classes (those who make at most \$50,000, and those who make more), it's clear most individuals do not make more than \$50,000. This can greatly affect **accuracy**, since we could simply say *"this person does not make more than \$50,000"* and generally be right, without ever looking at the data! Making such a statement would be called **naive**, since we have not considered any information to substantiate the claim. It is always important to consider the *naive prediction* for your data, to help establish a benchmark for whether a model is performing well. That been said, using that prediction would be pointless: If we predicted all people made less than \$50,000, *CharityML* would identify no one as donors. 

In [21]:
def rmsle(predicted, actual):
    return np.sqrt(np.nansum(np.square(np.log(predicted + 1) - np.log(actual + 1)))/float(len(predicted)))

In [23]:
from sklearn.metrics import make_scorer
scorer = make_scorer(rmsle, greater_is_better=False)

----
## PART 3: Model Implementation - XGBoost / ANN

In [28]:
def ann_model(dropout_rate=0.0, length = 557):
    # create model
    model = Sequential()
    model.add(Dense(378, input_dim=length, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(169, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))    
    model.add(Dense(84, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(172, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(85, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(45, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_logarithmic_error', optimizer='adam')
    return model

In [29]:
ann_regressor = KerasRegressor(build_fn=ann_model, epochs=30, batch_size=10, verbose=1)

/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:80: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  legal_params += inspect.getargspec(fn)[0]
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:80: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  legal_params += inspect.getargspec(fn)[0]
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:80: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  legal_params += inspect.getargspec(fn)[0]
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:80: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  legal_params += inspect.ge

In [30]:
ann_score = cross_val_score(ann_regressor, train_data_X, train_data_y, cv = 5, verbose = 10, scoring = scorer)
ann_score = np.mean(ann_score)

/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:80: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  legal_params += inspect.getargspec(fn)[0]
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:80: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  legal_params += inspect.getargspec(fn)[0]
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:80: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  legal_params += inspect.getargspec(fn)[0]
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:80: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  legal_params += inspect.ge

[CV]  ................................................................


/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:166: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  fn_args = inspect.getargspec(fn)[0]
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/utils/generic_utils.py:135: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  arg_spec = inspect.getargspec(cls.from_config)
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/utils/generic_utils.py:135: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  arg_spec = inspect.getargspec(cls.from_config)
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/utils/generic_utils.py:135: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  arg_spec = inspect.getar

Epoch 1/30
24376/24376 [==============================] - 24s - loss: 4.2384    
Epoch 2/30
24376/24376 [==============================] - 23s - loss: 0.2282    
Epoch 3/30
24376/24376 [==============================] - 22s - loss: 0.2119    
Epoch 4/30
24376/24376 [==============================] - 20s - loss: 0.2058    
Epoch 5/30
24376/24376 [==============================] - 20s - loss: 0.2014    
Epoch 6/30
24376/24376 [==============================] - 21s - loss: 0.1985    
Epoch 7/30
24376/24376 [==============================] - 24s - loss: 0.1958    
Epoch 8/30
24376/24376 [==============================] - 27s - loss: 0.1926    
Epoch 9/30
24376/24376 [==============================] - 23s - loss: 0.1913    
Epoch 10/30
24376/24376 [==============================] - 23s - loss: 0.1896    
Epoch 11/30
24376/24376 [==============================] - 23s - loss: 0.1876    
Epoch 12/30
24376/24376 [==============================] - 22s - loss: 0.1857    
Epoch 13/30
24376/24376 [

/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:166: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  fn_args = inspect.getargspec(fn)[0]


...

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 45.6min remaining:    0.0s
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:80: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  legal_params += inspect.getargspec(fn)[0]
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:80: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  legal_params += inspect.getargspec(fn)[0]
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:80: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  legal_params += inspect.getargspec(fn)[0]
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:80: DeprecationWarning: inspect.getargspec() is deprecated, 

Epoch 1/30
24377/24377 [==============================] - 21s - loss: 3.9517    
Epoch 2/30
24377/24377 [==============================] - 20s - loss: 0.2684    
Epoch 3/30
24377/24377 [==============================] - 21s - loss: 0.2489    
Epoch 4/30
24377/24377 [==============================] - 20s - loss: 0.2418    
Epoch 5/30
24377/24377 [==============================] - 21s - loss: 0.2376    
Epoch 6/30
24377/24377 [==============================] - 20s - loss: 0.2331    
Epoch 7/30
24377/24377 [==============================] - 20s - loss: 0.2312    
Epoch 8/30
24377/24377 [==============================] - 21s - loss: 0.2274    
Epoch 9/30
24377/24377 [==============================] - 21s - loss: 0.2251    
Epoch 10/30
24377/24377 [==============================] - 22s - loss: 0.2232    
Epoch 11/30
24377/24377 [==============================] - 22s - loss: 0.2212    
Epoch 12/30
24377/24377 [==============================] - 21s - loss: 0.2188    
Epoch 13/30
24377/24377 [

/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/wrappers/scikit_learn.py:166: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  fn_args = inspect.getargspec(fn)[0]


5860/6094 [===========================>..] - ETA: 0s[CV] ................................ , score=-0.593090, total=11.3min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 56.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 56.9min finished


In [31]:
print ("ANN without optimization will have rmsle score of : " + str(ann_score))

ANN without optimization will have rmsle score of : -0.590711383088


In [32]:
xgb_regressor = xgb(verbose = 1, random_state = 2017, n_jobs = -1)

In [33]:
xgb_score = cross_val_score(xgb_regressor, train_data_X, train_data_y, cv = 5, verbose = 10, scoring = scorer)
xgb_score = np.mean(xgb_score)

[CV]  ................................................................
[CV] ................................ , score=-0.606609, total=  49.7s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   49.7s remaining:    0.0s


[CV] ................................ , score=-0.431028, total=  52.5s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.7min remaining:    0.0s


[CV] ................................ , score=-0.481119, total=  58.3s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.7min remaining:    0.0s


[CV] ................................ , score=-0.458049, total=  46.8s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  3.5min remaining:    0.0s


[CV] ................................ , score=-0.426416, total=  50.7s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  4.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  4.3min finished


In [34]:
print ("XBoost without optimization will have rmsle score of : " + str(xgb_score))

XBoost without optimization will have rmsle score of : -0.480643999869


----
## PART 4: Model Optimization 

In [37]:
def AutoGridSearch(parameters, regressor, train_data_X, train_data_y):

    scorer = make_scorer(rmsle, greater_is_better=False)
#    while True:
    
    #Perform grid search on the classifier using 'scorer' as the scoring method
    grid_obj = GridSearchCV(estimator = regressor,
                               param_grid = parameters,
                               scoring = scorer,
                               cv = 5,
                               verbose=10,
                               n_jobs = -1)
    
    #Fit the grid search object to the training data and find the optimal parameters
    grid_fit = grid_obj.fit(train_data_X, train_data_y)
    
    #if(best_score < grid_fit.best_score_):
    best_score = grid_fit.best_score_

    best_parameters = grid_fit.best_params_
    
    best_estimator = grid_fit.best_estimator_
    
    print("Best: %f using %s" % (grid_fit.best_score_, grid_fit.best_params_))
    for params, mean_score, scores in grid_fit.grid_scores_:
        print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))
    
    return best_score, best_parameters, best_estimator

**ANN with parameter optimization**

In [ ]:
dropout_rate = [0.0, 0.001, 0.01]
ann_parameters = dict(dropout_rate=dropout_rate)
ann_score, best_parameters, best_ann_model = AutoGridSearch(ann_parameters,ann_regressor, train_data_X, train_data_y)

**XGBoost with parameter optimization**

In [44]:
xgb_parameters = {'learning_rate' : [0.7, 0.8, 1.0], 
         'min_child_weight' : [1, 3, 5],
         'max_depth' : [5, 7 ,9],
         'subsample' : [0.6, 0.8, 1.0]}

score, best_parameters, best_xgb_model = AutoGridSearch(xgb_parameters,xgb_regressor, train_data_X, train_data_y)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] learning_rate=0.7, max_depth=5, min_child_weight=1, subsample=0.6 
[CV] learning_rate=0.7, max_depth=5, min_child_weight=1, subsample=0.6 
[CV] learning_rate=0.7, max_depth=5, min_child_weight=1, subsample=0.6 
[CV] learning_rate=0.7, max_depth=5, min_child_weight=1, subsample=0.6 


/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


[CV]  learning_rate=0.7, max_depth=5, min_child_weight=1, subsample=0.6, score=-0.706416, total= 5.1min
[CV] learning_rate=0.7, max_depth=5, min_child_weight=1, subsample=0.6 


/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  
/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


[CV]  learning_rate=0.7, max_depth=5, min_child_weight=1, subsample=0.6, score=-0.548933, total= 5.3min
[CV]  learning_rate=0.7, max_depth=5, min_child_weight=1, subsample=0.6, score=-0.524764, total= 5.2min
[CV] learning_rate=0.7, max_depth=5, min_child_weight=1, subsample=0.8 
[CV]  learning_rate=0.7, max_depth=5, min_child_weight=1, subsample=0.6, score=-0.515202, total= 5.3min
[CV] learning_rate=0.7, max_depth=5, min_child_weight=1, subsample=0.8 
[CV] learning_rate=0.7, max_depth=5, min_child_weight=1, subsample=0.8 


...

[CV]  learning_rate=1.0, max_depth=9, min_child_weight=5, subsample=1.0, score=-0.611458, total= 3.1min


[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed: 523.0min finished


Best: -0.518587 using {'learning_rate': 0.7, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 1.0}
-0.559749 (0.074844) with: {'learning_rate': 0.7, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.6}
-0.534305 (0.066594) with: {'learning_rate': 0.7, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.8}
-0.518583 (0.069426) with: {'learning_rate': 0.7, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 1.0}
-0.551740 (0.058935) with: {'learning_rate': 0.7, 'max_depth': 5, 'min_child_weight': 3, 'subsample': 0.6}
-0.535549 (0.065882) with: {'learning_rate': 0.7, 'max_depth': 5, 'min_child_weight': 3, 'subsample': 0.8}
-0.526388 (0.060136) with: {'learning_rate': 0.7, 'max_depth': 5, 'min_child_weight': 3, 'subsample': 1.0}
-0.555443 (0.063707) with: {'learning_rate': 0.7, 'max_depth': 5, 'min_child_weight': 5, 'subsample': 0.6}
-0.531923 (0.066778) with: {'learning_rate': 0.7, 'max_depth': 5, 'min_child_weight': 5, 'subsample': 0.8}
-0.524696 (0.079370) with: {'learning_rate

/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)



-0.599722 (0.062366) with: {'learning_rate': 1.0, 'max_depth': 5, 'min_child_weight': 5, 'subsample': 0.8}
-0.574986 (0.056022) with: {'learning_rate': 1.0, 'max_depth': 5, 'min_child_weight': 5, 'subsample': 1.0}
-0.689422 (0.056905) with: {'learning_rate': 1.0, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.6}
-0.651526 (0.077273) with: {'learning_rate': 1.0, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.8}
-0.600641 (0.060578) with: {'learning_rate': 1.0, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 1.0}
-0.708391 (0.049081) with: {'learning_rate': 1.0, 'max_depth': 7, 'min_child_weight': 3, 'subsample': 0.6}
-0.655206 (0.059508) with: {'learning_rate': 1.0, 'max_depth': 7, 'min_child_weight': 3, 'subsample': 0.8}
-0.610521 (0.071830) with: {'learning_rate': 1.0, 'max_depth': 7, 'min_child_weight': 3, 'subsample': 1.0}
-0.721175 (0.073855) with: {'learning_rate': 1.0, 'max_depth': 7, 'min_child_weight': 5, 'subsample': 0.6}
-0.644466 (0.053486) with: {'learnin

In [ ]:
print (score)
best_xgb_parameters = {'learning_rate' : 0.8, 
         'min_child_weight' : 1,
         'max_depth' : 7,
         'subsample' : 0.8}

----
## PART 4: Result Comparsion with Simple Linear Regression

In [39]:
lr_regressor = lr()

In [40]:
lr_score = cross_val_score(lr_regressor, train_data_X, train_data_y, cv = 5, verbose = 10, scoring = scorer)
lr_score = np.mean(lr_score)
print ("Linear Regression without optimization will have rmsle score of : " + str(lr_score))

[CV]  ................................................................


/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s


[CV] ................................ , score=-0.266111, total=   2.6s
[CV]  ................................................................


/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.0s remaining:    0.0s


[CV] ................................ , score=-0.591429, total=   2.4s
[CV]  ................................................................


/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    7.2s remaining:    0.0s


[CV] ................................ , score=-0.685187, total=   2.2s
[CV]  ................................................................


/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    9.6s remaining:    0.0s


[CV] ................................ , score=-0.510681, total=   2.4s
[CV]  ................................................................
[CV] ................................ , score=-0.584983, total=   2.6s
Linear Regression without optimization will have rmsle score of : -0.527678379994


/Applications/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   12.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   12.2s finished


#### Results:

|     Rmsle Score   | Linear Regression | Deep Learning (ANN) | Gradient Boosting (XGBoost) |
| :------------: | :-----------------: | :---------------: | :-------------: | 
| Un-optimized Model |       -0.528        |      -0.591       |     -0.481      |
| Optimized Model       |       Not Applicable        |      0.0       |     0.0      |

In [ ]:
def submit(test_data, y_pred, filename, training, model, scores = 0):

    results = pd.DataFrame({
    'id' : test_data['id'].astype(np.int32),
    'price_doc' : y_pred
    })

    scoreStr = "{:.3f}".format(scores)
    
    if(training):
        results.to_csv("./training_" + model + "_" + filename + "_" + scoreStr + ".csv", index=False)
    else:
        results.to_csv("./test_" + model + "_" + filename + "_" + scoreStr + ".csv", index=False)
